In [1]:
import pandas as pd
import os
import glob
import re
import string
from typing import Iterable

In [2]:
data_path = '../data/'

In [3]:
exp_files = glob.glob(os.path.join(data_path + 'train_data/Expenses', '*.xlsx'))
exp_dfs = []
for filename in exp_files:
    exp_dfs.append(pd.read_excel(filename, sheet_name='Таблица1'))
expenses_data = pd.concat(exp_dfs, axis=0, ignore_index=True)
print(expenses_data.shape)

(23150, 11)


In [4]:
expenses_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23150 entries, 0 to 23149
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Итого                       23150 non-null  object        
 1   Год сметы                   23150 non-null  int64         
 2   Смета                       23150 non-null  object        
 3   Подразделение БР            23150 non-null  object        
 4   Код статьи                  23150 non-null  float64       
 5   Код вида расходов           23150 non-null  object        
 6   Наименование вида расходов  23150 non-null  object        
 7   Направление расходов        23150 non-null  object        
 8   Отнесено                    23150 non-null  float64       
 9   Конечный остаток            23150 non-null  float64       
 10  Дата проведения             23150 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)


In [5]:
unqie_values = expenses_data[expenses_data['Год сметы'] == 2020].apply(pd.Series.nunique)
unqie_values

Итого                          249
Год сметы                        1
Смета                           13
Подразделение БР                20
Код статьи                       1
Код вида расходов                5
Наименование вида расходов       5
Направление расходов           249
Отнесено                      2620
Конечный остаток              2620
Дата проведения                213
dtype: int64

In [6]:
expenses_data = expenses_data[expenses_data.columns.difference(unqie_values[unqie_values == 1].index)]

In [7]:
expenses_data[expenses_data['Конечный остаток'] != expenses_data['Отнесено']].shape[0]

0

In [8]:
expenses_data.drop(columns=['Конечный остаток', 'Смета'], inplace=True)

In [9]:
expenses_data['Код вида расходов'].apply(len).value_counts()

Код вида расходов
15    23150
Name: count, dtype: int64

In [10]:
expenses_data['Код вида расходов'].apply(lambda c: c[-4:] == '0000').value_counts()

Код вида расходов
True    23150
Name: count, dtype: int64

In [11]:
expenses_data['Код вида расходов'] = expenses_data['Код вида расходов'].apply(lambda c: c[:-4])

In [12]:
def remove_multiple_spaces(text: str):
    return re.sub(r'\s+', ' ', text)

In [13]:
expenses_data['Итого'] = expenses_data.Итого.apply(
    lambda addr: remove_multiple_spaces(addr)
)

In [14]:
expenses_data['Итого'] = expenses_data.apply(
    lambda exp: exp['Итого']\
        .replace('Итого по направлению - ', '')\
        .replace(exp['Наименование вида расходов'], '')\
        .replace(exp['Код вида расходов'], '')\
        .replace('ст.' + exp['Код вида расходов'][:-3], '')\
        .replace(exp['Код вида расходов'][:-3], '')\
        .replace('Расходы прошлого года', '')\
        .strip(), axis=1
)
expenses_data['Итого'] = expenses_data.Итого.apply(
    lambda addr: remove_multiple_spaces(addr)
)

In [15]:
rubbish_patterns = [
    'оплата',
    'расходы'
]

expenses_data[
    expenses_data.Итого.apply(
        lambda addr: any(patt in addr.lower() for patt in rubbish_patterns)
        )
    ].Итого.to_clipboard()

In [16]:
def remove_value_from_str(text: str, values: Iterable) -> str:
    for value in values:
        text = text.replace(value, '')
    return text


def remove_text_in_brackets(text: str) -> str:
    return re.sub("[\(\[].*?[\)\]]", "", text)


def remove_number_from_beginning(text: str):
    match = re.match(r'\d+', text)
    if match:
        result = text[len(match.group(0)):]
        return result
    else:
        return text

In [17]:
to_remove = [
    'Расходы на коммунальные услуги',
    'Оплата потребления газа',
    'Оплата за потребление газа',
    'Оплата за электроэнергию по тарифам',
    'Оплатата потребления электроэнергии',
    'Оплата за эектроэнергию',
    'Оплата за тепловую энергию',
    'Тепловая энергия',
    'Плата за технологическое присоед. к электрическим сетям',
    'Оплата потребления электроэнерги',
    'Оплата за потребление тепловой энергии',
    'Оплата водоотведения, канализации',
    'Оплата водоотведения',
    'Оплата водоотведения',
    'Оплата водоотведение',
    'Оплата за водоотведение',
    'Оплата за горячее водоснабжение',
    'Оплата горячее водоснабжение',
    'Оплата горячего водоснабжения',
    'Оплата горячево водоснабжения',
    'Оплата горячее водоснабжения',
    'Оплата горячего водоснабжение',
    'Оплата услуг за горячее водоснабжение',
    'Оплата за услуги горячего водоснабжения',
    'Оплата за услуги холодного водоснабжения',
    'Оплата услуг водоотведения',
    'Оплата за водоснабжение',
    'Оплата холодного водоснабжения',
    'Оплата холодное водоснабжение',
    'Оплата холодное водоснабжения',
    'Оплата за холодное водоснабжение',
    'Оплата водоснабжения',
    'Оплата канализации',
    'Оплата за канализацию',
    'Оплата за услуги водоотведения',
    'Канализация',
    'Водоснабжение холодная вода',
    'Холодное водоснабжение',
    'Холодная вода',
    'Горячее водоснабжение',
    'Горячее водоснабжения',
    'Водоснабжение горячая вода',
    'Водоотведение',
    'Другие расходы',
    'Оплата потребления эл.энергии',
    'Затраты на горячее водоснабжение',
    'Затраты на холодное водоснабжение',
    'Затраты на канализацию',
    'затраты на дизельное топливо',
    'Оплата за дизельное топливо',
    'Оплата за прочие виды топлиа (уголь, дрова и т.д.)',
    'прошлых лет, выявленные в текущем году',
    'признанные после отчетной даты',
    'по тарифам',
    'Расходы по оплате отопления',
    'Расходы по приобретению дизельного топлива',
    'Расходы по оплате водоотведения',
    'Расходы по оплате холодного водоснабжения',
    'Расходы по оплате услуг за холодное водоснабжение',
    'расходы по оплате услуг за холодное водоснабжение',
    'Плата за сброс сточных вод',
    'Плата за негативное воздействие на централ.систему водоотведения',
    'Плата за негативное воздействие на центральные системы водоотведения',
    'Осн.здание.',
    'квартира',
    'ГАРАЖНОЕ помещение',
    'ГАРАЖ-БОКС',
    'ГАРАЖ',
    'Гаражный бокс',
    'Гараж',
    'гаражи, боксы № 9, 10',
    '(бокс №1, №2)',
    '№ 5',
    'гаражный бокс',
    'гараж',
    'СКЛАД',
    'Склады хоз.зоны',
    'Склад',
    'склад',
    'Админ. здание',
    'ЗДАНИЕ',
    'здание',
    'здания',
    'Здание',
    'зд.',
    'Отделения',
    'Отд',
    'пристройка',
    'помещение объект № 2',
    'СУЖЕБНОЕ ПОМЕЩЕНИЕ',
    'служебное',
    'Служебное помещение',
    'нежилые помещения',
    'нежилое помещение',
    'Контрольно-пропускной пункт',
    'Контрольно-пропускной',
    'функц. помещение',
    'Убежище',
    'Здарвпункт',
    'Врачебный здравпункт',
    '(здравпункт)',
    'СТОЛ',
    'Столовая',
    'столовая',
    'БУФЕТ',
    'Буфет',
    'буфет',
    'ПРЭ',
    'Спальный корпус',
    'Спальный',
    'Пристройка к Спальному корпусу',
    'Присторойка к спальному корпусу',
    'Пристройка спального корпуса',
    'Пристройка спального',
    'Клуб',
    'Лечебный корпус',
    'Лечебный',
    'Лечебное отделение',
    'Лыжная станция',
    'Теплица',
    'Контора хоз.части',
    'База РСГ',
    'База МТО',
    'База РСО',
    'Пляж',
    'Сторожка пляжа',
    'Котельная',
    'Здравпункт',
    'шахт.канализ.станция',
    'шахтн. канализац. станция',
    'спец.подвал',
    'сопка',
    'МЗССС',
    'Помещение объект',
    'Помещение Объект',
    '№ 2', '№2',
    'Пристройка к зданию',
    'Пристройка',
    'Административное',
    'ст.1341100',
    '134.11100.03000',
    '134,1000,02',
    '132.0112',
    'жилой подъезд',
    'на 10 автомобилей',
    'на 10 а/м',
    'Плата за технологическое присоединение к электрическим сетям (в случаях, не связанным со строительством,реконстр. и кап.ремонтом',
    'Плата за технологическое присоединение к электрическим сетям',
    'дизель-генераторная установка',
    '0000',
    'Администр',
    '№ 2', '№ 1',
    'часть жилого',
    'ЗО',
    'ДГУ',
    '«Технологическое присоединение»',
    'ЕАО',
    'Расходы по оплате',
    'электроэнергию',
    'ХВС',
    'ГВС',
    'Общежитие',
    'Хаб. коммунальных услуг',
    'РКЦ',
    '02',
    '№'
]

expenses_data['Итого'] = expenses_data.Итого.apply(
    lambda addr: remove_text_in_brackets(remove_value_from_str(addr, to_remove))
)

puncts_translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

expenses_data['Итого'] = expenses_data.Итого.apply(
    lambda addr: remove_multiple_spaces(addr.translate(puncts_translator)).lower().strip()
)
expenses_data['Итого'] = expenses_data.Итого.apply(remove_number_from_beginning).str.strip()

In [18]:
expenses_data = expenses_data[expenses_data.Итого != ''].reset_index(drop=True)

In [19]:
expenses_data['Дата проведения'] = pd.to_datetime(expenses_data['Дата проведения'], yearfirst=True)
expenses_data['Код вида расходов'] = expenses_data['Код вида расходов'].astype('category')
expenses_data['Отнесено'] = expenses_data['Отнесено'].astype(float)

In [35]:
# pd.DataFrame(expenses_data.Итого.unique(), columns=['expenses_address']).to_clipboard()

In [30]:
# expenses_data.to_feather(data_path + 'preprocessed/Expenses/expenses_preprocessed.frt')

In [3]:
expenses_data = pd.read_feather(data_path + 'preprocessed/Expenses/expenses_preprocessed.frt')

In [4]:
expenses_data['date'] = expenses_data['Дата проведения'] + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)

target_data = expenses_data.groupby(['Итого', 'date'], as_index=False).Отнесено.sum()
print(target_data.shape)

(6521, 3)


In [5]:
geocoder_res = pd.read_feather(data_path + 'geo/geocoder_results_final.frt')

In [6]:
target_data = target_data.merge(
    geocoder_res[['raw_string', 'geocoder_address', 'geocoder_lat', 'geocoder_lon']],
    left_on='Итого', right_on='raw_string', how='left'
)

In [7]:
target_data = target_data[target_data.geocoder_address.notna()].reset_index(drop=True)

In [8]:
target_data.drop(columns=['raw_string'], inplace=True)
target_data.rename(columns={'Отнесено': 'paid', 'Итого': 'raw_address'}, inplace=True)

In [9]:
# target_data.to_feather(data_path + 'final/expenses_by_object_and_month.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
